We import `netgen.gui` to open the netgen GUI for displaying the ferromagnet. 
We then define the geometry, and draw the box. 
The refinement of the mesh is controlled by `H_MAX` in `ngmesh`.

In [ ]:
from netgen.csg import *
from ngsolve import *
from ngsolve.utils import grad, Grad  # If I don't import these explicitly, VSCode reads them as missing.
import netgen.gui  # this opens up the netgen ui
import Magnetisation_Functions
T_MAX = 10.0  # The maximum time for the simulation
ALPHA = 1  # Dissipative constant in the LLG equation.
THETA = 0.75 # Should be strictly above 1/2 for unconditional stability
K = 0.1  # TIME STEP
KAPPA = 1  # Determines the relative strength of the elastic vs. magnetic parts.
H_MAX = 1  # Determines how fine the mesh should be.

In [ ]:
def MakeGeometry():  # this makes a box, with labelled faces
    geometry = CSGeometry()
    left  = Plane(Pnt(0,0,0), Vec(-1,0,0)).bc("left")
    right = Plane(Pnt(1,1,1), Vec( 1,0,0)).bc("right")
    front = Plane(Pnt(0,0,0), Vec(0,-1,0)).bc("front")
    back  = Plane(Pnt(1,1,1), Vec(0, 1,0)).bc("back")
    bot   = Plane(Pnt(0,0,0), Vec(0,0,-1)).bc("bot")
    top   = Plane(Pnt(1,1,1), Vec(0,0, 1)).bc("top")

    cube = left * right * front * back * bot * top
    geometry.Add (cube)
    #cube = OrthoBrick(Pnt(0,0,0), Pnt(1,1,1))
    geometry.Add(cube)
    return geometry



ngmesh = MakeGeometry().GenerateMesh(maxh=H_MAX)
# ngmesh.Save("cube.vol")
mesh = Mesh(ngmesh)
Draw(mesh)

In [ ]:
  # this helps me figure out which sides are which.
#fes_SCALAR = H1(mesh, order=1)
#myScalars = {"left":10, "right":10, "front":0, "back":0, "bot":5, "top":5}
#SCALAR_gfu = CoefficientFunction([myScalars[val] for val in mesh.GetBoundaries()])

#Draw(SCALAR_gfu, mesh, "SCALAR_gfu")

In [ ]:
fes_mag = VectorH1(mesh, order=1)  # the finite element space for the magnetisation m_h^i
fes_disp = VectorH1(mesh, order=1, dirichlet="bot")  # the finite element space for the displacement u_h^i

print(f"mag_ndof={fes_mag.ndof}, disp_ndof={fes_disp.ndof},\n, dispfree_ndof={fes_disp.FreeDofs()}")
mag_gfu = GridFunction(fes_mag)
disp_gfu = GridFunction(fes_disp)

In [ ]:
v = fes_mag.TrialFunction()
phi = fes_mag.TestFunction()
u = fes_disp.TrialFunction()
psi = fes_disp.TestFunction()

mag_gfu = Magnetisation_Functions.give_random_magnetisation(mag_gfu)

a_mag = BilinearForm(fes_mag)
a_mag += ALPHA*InnerProduct(v, phi)*dx
a_mag += THETA*K*InnerProduct(Grad(v), Grad(phi))*dx
a_mag += InnerProduct(Cross(mag_gfu,v), phi)*dx
a_mag.Assemble()

f_mag = LinearForm(fes_mag)
f_mag += -InnerProduct(Grad(mag_gfu), Grad(phi))*dx
#f_mag +=  # This is where we will put the magnetostrictive contribution.
f_mag.Assemble()


In [ ]:
B_T = Magnetisation_Functions.build_tangent_plane_matrix_transpose(mag_gfu)

Mag_Block = BlockMatrix([a_mag.mat, B_T.data],[B_T.data.T,None])

In [ ]:
#print(mag_gfu.vec)
Draw(mag_gfu)
print(Magnetisation_Functions.build_tangent_plane_matrix_transpose(mag_gfu))
